In [1]:
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# load data
train = pd.read_csv('./train.csv').iloc[:, 1:]
test = pd.read_csv('./test.csv')

# split data into X and y
dfX = train.iloc[:,:-1]
dfy = train.iloc[:,[-1]]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfX, dfy)

## 데이터 탐색

In [12]:
#train.tail()
test.tail()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
565887,581008,2396,153,20,85,17,108,240,237,118,...,0,0,0,0,0,0,0,0,0,0
565888,581009,2391,152,19,67,12,95,240,237,119,...,0,0,0,0,0,0,0,0,0,0
565889,581010,2386,159,17,60,7,90,236,241,130,...,0,0,0,0,0,0,0,0,0,0
565890,581011,2384,170,15,60,5,90,230,245,143,...,0,0,0,0,0,0,0,0,0,0
565891,581012,2383,165,13,60,4,67,231,244,141,...,0,0,0,0,0,0,0,0,0,0


* train datasets 15119개
* test datasets 565891개
* 독립변수 12개
* 종속변수 1개 Cover_type

## 변수 설명

* Elevation - Elevation in meters (높이 /meter 단위)
* Aspect - Aspect in degrees azimuth (방위각)
* Slope - Slope in degrees (기울기 각도 /도 단위)
* Horizontal_Distance_To_Hydrology - Horz Dist to nearest surface water features (수원과의 수평거리)
* Vertical_Distance_To_Hydrology - Vert Dist to nearest surface water features (수원과의 수직거리)
* Horizontal_Distance_To_Roadways - Horz Dist to nearest roadway (길가와의 수평거리)
* Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice (오전 9시의 차양 / 0~255)
* Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice (정오시의 차양/ 0~255)
* Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice (오후 9시의 차양/ 0~255)
* Horizontal_Distance_To_Fire_Points - Horz Dist to nearest wildfire ignition points (야생 산불 발화지점과의 수평거리)
* Wilderness_Area (4 binary columns, 0 = absence or 1 = presence) - Wilderness area designation (황야 지대 /4종류 ) in Roosevelt National Forest of northern Colorado
* Soil_Type (40 binary columns, 0 = absence or 1 = presence) - Soil Type designation (토양 종류 / 40종류)
* 토양종류와 황야 지대 카테고리별 설명은 https://www.kaggle.com/c/forest-cover-type-prediction/data 참조

## 종속 변수

* Cover_Type (7 types, integers 1 to 7) - Forest Cover Type designation- (산림 유형 / 7종류) (the predominant kind of tree cover)
* 1 - Spruce/Fir
* 2 - Lodgepole Pine
* 3 - Ponderosa Pine
* 4 - Cottonwood/Willow
* 5 - Aspen
* 6 - Douglas-fir
* 7 - Krummholz

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 55 columns):
Elevation                             15120 non-null int64
Aspect                                15120 non-null int64
Slope                                 15120 non-null int64
Horizontal_Distance_To_Hydrology      15120 non-null int64
Vertical_Distance_To_Hydrology        15120 non-null int64
Horizontal_Distance_To_Roadways       15120 non-null int64
Hillshade_9am                         15120 non-null int64
Hillshade_Noon                        15120 non-null int64
Hillshade_3pm                         15120 non-null int64
Horizontal_Distance_To_Fire_Points    15120 non-null int64
Wilderness_Area1                      15120 non-null int64
Wilderness_Area2                      15120 non-null int64
Wilderness_Area3                      15120 non-null int64
Wilderness_Area4                      15120 non-null int64
Soil_Type1                            15120 non-null int64
Soil_T

### 데이터 전처리

## Cross Validation (K-fold)

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [6]:
knn_clf = KNeighborsClassifier(n_neighbors = 10)
scoring = 'accuracy'
score = cross_val_score(knn_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Py

[ 0.77579365  0.78968254  0.77380952  0.77513228  0.78571429  0.80753968
  0.77910053  0.79497354  0.77579365  0.78240741]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [ ]:
# kNN Score
round(np.mean(score)*100, 2)

## XGBoost

In [10]:
xgb_clf =  XGBClassifier(max_depth = 2)
scoring = 'accuracy'
score = cross_val_score(xgb_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/preproce

[ 0.72156085  0.73346561  0.71891534  0.7037037   0.7255291   0.73478836
  0.72949735  0.73544974  0.7281746   0.73346561]


In [9]:
# xgb Score
round(np.mean(score)*100, 2)

84.349999999999994

## ExtraTree

In [8]:
extra_clf = ExtraTreesClassifier()
scoring = 'accuracy'
score = cross_val_score(extra_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Pytho

[ 0.84259259  0.83267196  0.84126984  0.84060847  0.83994709  0.86507937
  0.8452381   0.83068783  0.84060847  0.85648148]


In [ ]:
# extra Score
round(np.mean(score)*100, 2)

## GradientBoost

In [ ]:
gboost_clf = GradientBoostingClassifier()
scoring = 'accuracy'
score = cross_val_score(gboost_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# gradientboost Score
round(np.mean(score)*100, 2)

## Decision Tree

In [ ]:
dt_clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(dt_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# decision tree Score
round(np.mean(score)*100, 2)

In [ ]:
clf2 = DecisionTreeClassifier(max_depth=4)
clf2.fit(dfX, dfy)
print(clf2.feature_importances_)

## Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=10)
scoring = 'accuracy'
score = cross_val_score(rf_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Random Forest Score
round(np.mean(score)*100, 2)

## Naive Bayes

In [ ]:
nb_clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(nb_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Naive Bayes Score
round(np.mean(score)*100, 2)

## SVM

In [ ]:
#svc_clf = SVC(probability=True) #for softvoting
svc_clf = SVC()
scoring = 'accuracy'
score = cross_val_score(svc_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

In [ ]:
#hard voting
voting_clf = VotingClassifier(estimators=[('knn', knn_clf),('xgb',xgb_clf),('extra', extra_clf),
                                          ('gdboost',gboost_clf),('DT',dt_clf),('rf',rf_clf),
                                          ('nb',nb_clf),('svc',svc_clf)], voting='hard')
scoring = 'accuracy'
score = cross_val_score(voting_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (knn_clf, xgb_clf, extra_clf, gboost_clf, dt_clf, rf_clf, nb_clf, svc_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

### 성능 좋은 것만 골라서 다시 voting

In [ ]:
#hard voting2
voting_clf = VotingClassifier(estimators=[('extra', extra_clf),('gdboost',gboost_clf),
                                          ('DT',dt_clf),('rf',rf_clf)], voting='hard')
                                    
scoring = 'accuracy'
score = cross_val_score(voting_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (extra_clf, gboost_clf, dt_clf, rf_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

## Voting

In [ ]:
#soft voting
voting_clf = VotingClassifier(estimators=[('knn', knn_clf),('xgb',xgb_clf),('extra', extra_clf),
                                          ('gdboost',gboost_clf),('DT',dt_clf),('rf',rf_clf),
                                          ('nb',nb_clf),('svc',svc_clf)], voting='soft')
scoring = 'accuracy'
score = cross_val_score(voting_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

## Testing

In [ ]:
final_clf = VotingClassifier(estimators=[('extra', extra_clf),('gdboost',gboost_clf),
                                          ('DT',dt_clf),('rf',rf_clf)], voting='hard')
final_clf.fit(dfX, dfy)

test_data = test.drop("Id", axis=1).copy()
prediction = clf.predict(test_data)

In [ ]:
submission = pd.DataFrame({
        "Id": test["Id"],
        "Cover_type": prediction
    })

submission.to_csv('submission2.csv', index=False)

In [ ]:
submission = pd.read_csv('submission2.csv')
submission.head()

In [ ]:
#soft voting2
voting_clf = VotingClassifier(estimators=[('extra', extra_clf),('gdboost',gboost_clf),
                                          ('DT',dt_clf),('rf',rf_clf)], voting='soft')
                                    
scoring = 'accuracy'
score = cross_val_score(voting_clf, dfX, dfy, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (extra_clf, gboost_clf, dt_clf, rf_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))